In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# https://github.com/matplotlib/ipympl/blob/main/docs/examples/full-example.ipynb
# https://kapernikov.com/ipywidgets-with-matplotlib/
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
# https://www.statology.org/pandas-subplots/

In [ ]:
def read(file):
    _df = pd.read_csv(file, date_format='%d.%m.%Y, %H:%M:%S', index_col ="Timestamp")
    _df['energy'] = _df['Energy in Wm (MRAM)'] / 60 / 1000
    return _df['energy']

folder_path = './data'
files = os.listdir(folder_path)
frames = map(read, map(lambda f: folder_path + '/' + f, files))
df = pd.concat(frames)

In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

date_format = '%Y-%m-%d'
yesterday = datetime.now() + timedelta(days=1)
past = yesterday - timedelta(weeks=1, days=1)

start_date = past.strftime(date_format)
end_date = yesterday.strftime(date_format)
resample = '1D'
 
from_picker = widgets.DatePicker(
    description='From',
    value=datetime.strptime(start_date, date_format),
    disabled=False
)
to_picker = widgets.DatePicker(
    description='To',
    value=datetime.strptime(end_date, date_format),
    disabled=False
)
resample_picker = widgets.Dropdown(
    description='Resample',
    value=resample, 
    options=['15T', '30T', '1H', '6H', '12H', '1D', '1W'], 
)
output_detail = widgets.Output()
output_plot = widgets.Output()

# set up plot
with output_plot:
    fig, ax = plt.subplots(constrained_layout=True, figsize=(20, 10))
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

def filter_df(df, start_date, end_date, resample):
    _df = df.loc[(df.index >= pd.Timestamp(start_date)) & (df.index <= pd.Timestamp(end_date))]
    return _df.resample(resample).sum()

def draw_df(df):
    ax.clear()
    ax.autoscale()
    ax.set_title('PV Energy')
    ax.set_xlabel('Timestamp')
    ax.set_ylabel('Energy [kWh]')
    ax.plot(df, color='C0')

def print_df_details(df):
    median = df.median()
    mean = df.mean()
    std = df.std()
    min = df.min()
    max = df.max()
    output_detail.clear_output()
    with output_detail:
        summary = pd.DataFrame({
            'Median': [median],
            'Mean': [mean],
            'Standard Deviation': [std],
            'Minimum': [min],
            'Maximum': [max]
        })
        print(summary
              .transpose()
              .to_string(header=False)
        )

def update(value):
    global start_date
    global end_date
    global resample

    description = value.owner.description
    if description == 'From':
        start_date = str(value.new)
    elif description == 'To':
        end_date = str(value.new)
    elif description == 'Resample':
        resample = str(value.new)

    _df = filter_df(df, start_date, end_date, resample)
    print_df_details(_df)
    draw_df(_df)

_df = filter_df(df, start_date, end_date, resample)
print_df_details(_df)
draw_df(_df)

# connect callbacks and traits
from_picker.observe(update, 'value')
to_picker.observe(update, 'value')
resample_picker.observe(update, 'value')

controls = widgets.VBox([from_picker, to_picker, resample_picker])
header = widgets.HBox([controls, output_detail])
header.layout = widgets.Layout(
        width='80%',
        display='flex',
        justify_content='space-around',
        align_items='center'
)
widgets.VBox([header, output_plot])